# Phonon computations

This is a quick sketch how to run a simple phonon calculation using DFTK.

> **Preliminary implementation**
>
> Practical phonon computations have only seen rudimentary testing as of now.
> As of now we do not yet recommend relying on this feature for production
> calculations. We appreciate any issues, bug reports or PRs.

First we run an SCF calculation.

In [1]:
using AtomsBuilder
using DFTK
using PseudoPotentialData

pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
model  = model_DFT(bulk(:Si); pseudopotentials, functionals=LDA())
basis  = PlaneWaveBasis(model; Ecut=10, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);
nothing  # hide

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.916437313755                   -0.69    5.0    119ms
  2   -7.921207574410       -2.32       -1.52    1.0   50.8ms
  3   -7.921403957151       -3.71       -2.49    1.4   54.3ms
  4   -7.921440978220       -4.43       -2.82    2.8   84.3ms
  5   -7.921441574576       -6.22       -2.97    1.0   51.2ms
  6   -7.921442007990       -6.36       -4.49    1.0   51.2ms
  7   -7.921442021709       -7.86       -4.54    2.8   77.0ms
  8   -7.921442022126       -9.38       -5.22    1.0   54.9ms
  9   -7.921442022138      -10.94       -5.37    2.0   61.5ms
 10   -7.921442022143      -11.25       -5.84    1.0   54.7ms
 11   -7.921442022144      -12.21       -6.54    1.1   54.1ms
 12   -7.921442022144      -13.35       -6.88    1.9   63.8ms
 13   -7.921442022144      -14.57       -7.64    1.1   53.7ms
 14   -7.921442022144   +    -Inf       -8.09    1.9   66.0ms


Next we compute the phonon modes at the q-point `[1/4, 1/4, 1/4]`.

In [2]:
scfres = DFTK.unfold_bz(scfres)
phret_q0 = @time DFTK.phonon_modes(scfres; q=[0.25, 0.25, 0.25]);
nothing  # hide

Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      73.0   2.60s  Non-interacting
   1        0       1        0.31     52.7   22.4s  
   2        0       2       -0.52     47.2   1.43s  
   3        0       3       -2.40     41.8   1.27s  
   4        0       4       -4.46     29.9   982ms  
   5        0       5       -6.46     17.7   711ms  
   6        0       6       -8.91      4.0   387ms  
   7        1       1       -7.86     55.6   1.99s  Restart
   8        1       2       -9.02      5.5   444ms  
                                      72.9   1.89s  Final orbitals
Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      73.0   2.08s  Non-interacting
   1        0       1        0.31     52.7   1.60s  
   2        0       2       -0.52     47.2   1.38s  
   3

These are the final phonon frequencies:

In [3]:
phret_q0.frequencies

6-element Vector{Float64}:
 -0.0034981553621156766
 -0.0029774511617812957
 -0.00297745116137843
  0.004302230618219201
  0.00430223061843963
  0.004353201749930729